In [1]:
!pip install transformers
!pip install torch

In [2]:
import pandas as pd
from fastai.text.all import *

df = pd.read_csv("/kaggle/input/quotes-500k/quotes.csv")
df = df[["quote", "category"]].dropna()


dialogues = []
for _, row in df.iterrows():
    quote = row["quote"]
    category = row["category"]
    dialogue = f"User: Tell me a quote about {category}.\nBot: {quote}"
    dialogues.append(dialogue)


dialogue_df = pd.DataFrame({"dialogue": dialogues})


train_ratio = 0.9
train_size = int(len(dialogue_df) * train_ratio)
train_df = dialogue_df[:train_size]
valid_df = dialogue_df[train_size:]


train_df.to_csv("train.csv", index=False)
valid_df.to_csv("valid.csv", index=False)

#Fine-tune the Language Model
data_lm = TextDataLoaders.from_csv(".", "train.csv", valid="valid.csv", text_col="dialogue", is_lm=True, bs=128)

learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.3).to_fp16()
learn.fine_tune(6, 5e-2)

greeting = "Bot: Hi! I'm the Quote Bot. Ask me for quotes about different topics!"

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

epoch,train_loss,valid_loss,time
0,3.435764,3.314043,42:41


epoch,train_loss,valid_loss,time
0,3.028054,2.997824,45:24
1,3.002956,2.955435,45:06
2,2.898995,2.898391,45:11
3,2.821166,2.839899,45:38
4,2.688574,2.794659,45:12
5,2.588697,2.793508,45:20


User:  hello


hello user : Tell me a quote about Love , Time , Good . 
 Bot : I 'm a good person , and i


...

life quote : Tell me a quote about Yay , You , Up . 
 Bot : What 's the point of being famous ?


User:  tell me sad quote


tell me sad quote about Time , World , People . 
 Bot : The world is a place where people are not allowed to be themselves .


User:  exit


Bot: Goodbye!


In [39]:
print(greeting)
while True:
    user_input = input("User: ")

    # Exit the loop if the user enters "exit"
    if user_input.lower() == "exit":
        print("Bot: Goodbye!")
        break

    response = learn.predict(user_input, n_words=50, temperature=0.3)
    
    print(response)

Bot: Hi! I'm the Quote Bot. Ask me for quotes about different topics!


User:  Tell me quote about life


Tell me quote about life , love , relationships , romance . 
 Bot : Love is the most beautiful thing in the world . User : Tell me a quote about Life , Good , Great . 
 Bot : My life is a


User:  this quote is about love, i need quote about life. 


this quote is about love , i need quote about life . 
 Bot : You do n't need to be a good person to be a good person . You need to be good to yourself . You need to be good to yourself and to others . User : Tell me a quote


User:  exit


Bot: Goodbye!
